In [1]:
from os import walk
import json
import re
from tqdm import tqdm
from utils import pmi, pmi_to_dict_adj_dict, docs_distribution, new_distribution, OptimizedIncrementalPMI
from scoring import compute_scores
import time
import pandas as pd
from textCleaning import get_file_names, extract_year_ipc
from sklearn.feature_extraction.text import CountVectorizer
import numpy as np
import utils
from utils import docs_distribution, new_distribution, combine_columns, OptimizedIncrementalPMI
import copy
from novelty import ClusterKS

import importlib
importlib.reload(utils)
from utils import docs_distribution, new_distribution, combine_columns, OptimizedIncrementalPMI
import scoring
importlib.reload(scoring)
from scoring import compute_scores
import novelty
importlib.reload(novelty)
import surprise
importlib.reload(surprise)

c:\Users\edgar\OneDrive\Bureau\Ecole\HEC\A24\BrevetNLP\.conda\Lib\site-packages\spacy\util.py:910: UserWarning: [W095] Model 'en_core_web_sm' (3.8.0) was trained with spaCy v3.8.0 and may not be 100% compatible with the current version (3.7.5). If you see errors or degraded performance, download a newer compatible model or retrain your custom model with the current spaCy version. For more details and available updates, run: python -m spacy validate
  warnings.warn(warn_msg)
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\edgar\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


<module 'surprise' from 'c:\\Users\\edgar\\OneDrive\\Bureau\\Ecole\\HEC\\A24\\BrevetNLP\\PatentNovelty\\novelty\\surprise.py'>

In [2]:
w_size = 3
path = "/home/edgarlanoue/data/csvCleaned/" #"C:/Users/edgar/OneDrive/Bureau/Ecole/HEC/A24/BrevetNLP/exemple données/cleaned/"
pathOutput = "/home/edgarlanoue/metrics/" #"C:/Users/edgar/OneDrive/Bureau/Ecole/HEC/A24/BrevetNLP/exemple données/metrics/"
path = "C:/Users/edgar/OneDrive/Bureau/Ecole/HEC/A24/BrevetNLP/exemple données/cleaned/"
pathOutput = "C:/Users/edgar/OneDrive/Bureau/Ecole/HEC/A24/BrevetNLP/exemple données/metrics/"

useClusters=True
ipcList = ["H01L"]
yearList = ["2016"]
tE_cols = ["claims"]
base_cols = ["claims", "background"]

In [3]:
# def measureNov(path, pathOutput, tE_cols, base_cols, w_size, useClusters, yearList, ipcList):
start_time = time.time()  # Record the starting time
if yearList == [] or ipcList == []:
    tE_names = get_file_names(path+"tE")
    KS_names = get_file_names(path+"KS")
    ES_names = get_file_names(path+"ES")

    tE_set = set([extract_year_ipc(string) for string in tE_names])
    KS_set = set([extract_year_ipc(string) for string in KS_names])
    ES_set = set([extract_year_ipc(string) for string in ES_names])

    assert(tE_set==KS_set==ES_set)

    yearList = sorted(list(set([year for year, ipc in list(tE_set)])))
    ipcList = list(set([ipc for year, ipc in list(tE_set)]))
    print(yearList)
    print(ipcList)

for year in yearList:
    for ipc in ipcList:
        print(f"{year}")
        print(f"    {ipc}")
        tE = pd.read_csv(path+f"tE/{year}_{ipc}_tE_cleaned.csv")
        KS = pd.read_csv(path+f"KS/{year}_{ipc}_KS_cleaned.csv")
        ES = pd.read_csv(path+f"ES/{year}_{ipc}_ES_cleaned.csv")

        ### Transforming KB into distribution
        application_number = tE["application_number"]
        label = tE["label"]

        tE=combine_columns(tE, tE_cols) #[:10000]
        KS=combine_columns(KS, base_cols) #[:100000]
        ES=combine_columns(ES, base_cols) #[:100000]

        # Création pmi ES
        # current_time = time.time()  # Get the current time
        # elapsed_time = current_time - start_time
        # start_time = current_time
        # print(elapsed_time)

        # print("ES")
        base_texts = [word for text in tqdm(ES) for word in text.split()]
        ES_PMI = OptimizedIncrementalPMI(3)
        ES_PMI.update(base_texts)
        instance_ES_pmi = ES_PMI.compute_pmi()
        base_bigram_set = set(instance_ES_pmi.keys())
        
        dict_ES_pmi = pmi_to_dict_adj_dict(instance_ES_pmi)
        # print("ES finidhsed")
        
        # # current_time = time.time()  # Get the current time
        # # elapsed_time = current_time - start_time
        # # start_time = current_time
        # # print(elapsed_time)

        # # print("KS")
        # KS_matrix, KS_dist, KS_Count_matrix = docs_distribution(baseSpace=KS, tE=tE)
        # KS_size = list(range(KS_matrix.shape[0]))
        # # print("KS finished")

        # # current_time = time.time()  # Get the current time
        # # elapsed_time = current_time - start_time
        # # start_time = current_time
        # # print(elapsed_time)

        # # print("Cluster")
        # if useClusters==True:
        #     KSClusterDiff1000 = ClusterKS(list_know_P=KS_matrix, new_Q= None, N=100, nbPtsPerCluster=1000)
        #     KSClusterDiff1000.clusterKS()
        # # print("Cluster finished")    
        # # ES_size = list(range(ES_matrix.shape[0]))
        
        # # current_time = time.time()  # Get the current time
        # # elapsed_time = current_time - start_time
        # # start_time = current_time
        # # print(elapsed_time)
        
        # ## We set to 0 the distance here for each recipe -- difference needs to estimate distance between all points. 
        # #This serves as optim to not calculate for each varaitions but only once since it is the same distance for all KB
        # # print('train variations for recipe {} done'.format(recette), ' | train variation size : ', len(train_recettes))
        
        # neighborhood_distance  = 0.
        # new_ratio_vec = []
        # new_bin_vec = []
        # uniq_ratio_vec = []
        # uniq_bin_vec = []
        # diff_ratio_vec = []
        # diff_bin_vec = []
        # neighborhood_distance_vec = []
        # surpDiv_ratio_vec = []
        # surpDiv_bin_vec = []
        # # mean100_vec = []
        
        # for i in len(tE):
        #     # print("new KS dist (with 1 toEval)")
        #     select_variation = KS_size + [len(KS_size)+i]
        #     NewKS_dist, variation_dist = new_distribution(KS_Count_matrix, select_variation)
            
        #     # print("new ES pmi (with 1 toEval)")
        #     # # instance_ES_updated_pmi = copy.deepcopy(incremental_pmi_ES_only) # reprise du pmi du ES seulement
        #     # new_pmi_instance = OptimizedIncrementalPMI(window_size = w_size)
        #     # update_text = [word for text in tqdm(tE[i]) for word in text.split()] 
        #     # new_pmi_instance.update(update_text)
        #     # new_pmi = new_pmi_instance.compute_pmi()

        #     baseTexts_update = [tE[i]]
        #     update_text = [word for text in (baseTexts_update) for word in text.split()]
        #     new_pmi = OptimizedIncrementalPMI(window_size=3)
        #     new_pmi.update(update_text)
        #     newpmi_PMI = new_pmi.compute_pmi()
            
        #     # print("compute scores")
        #     results = compute_scores(KB_matrix=KS_matrix, KB_dist=KS_dist, NewKB_dist=NewKS_dist, variation_dist=variation_dist, 
        #                                 EB_PMI=instance_ES_pmi, base_bigram_set=base_bigram_set, dict_know_pmi=dict_ES_pmi, New_EB_PMI=newpmi_PMI,
        #                                 neighbor_dist=neighborhood_distance, useClusters=True, KSCluster=KSClusterDiff1000)

        #     new_ratio = results[0]
        #     new_bin = results[1]

        #     uniq_ratio = results[2]
        #     uniq_bin = results[3]

        #     diff_ratio = results[4]
        #     diff_bin = results[5]
        #     neighborhood_distance = results[6] #update neighborhood distance so it isn't reset
        #     # mean100 = results[7]

        #     # surpNew_ratio = results[7]
        #     # surpNew_bin = results[8]

        #     surpDiv_ratio = results[8]
        #     surpDiv_bin = results[9]

        #     new_ratio_vec.append(new_ratio)
        #     new_bin_vec.append(new_bin)

        #     uniq_ratio_vec.append(uniq_ratio)
        #     uniq_bin_vec.append(uniq_bin)

        #     diff_ratio_vec.append(diff_ratio)
        #     diff_bin_vec.append(diff_bin)
        #     neighborhood_distance_vec.append(neighborhood_distance)
        #     # mean100_vec.append(mean100)

        #     surpDiv_ratio_vec.append(surpDiv_ratio)
        #     surpDiv_bin_vec.append(surpDiv_bin)

        #     if i % 1000 == 0:
        #         current_time = time.time()  # Get the current time
        #         elapsed_time = current_time - start_time  # Calculate elapsed time
        #         if i % 1000 == 0:
        #             print(f"{i} on {len(tE)}, time since last print: {elapsed_time:.2f} seconds")
        #         start_time = current_time  # Reset the start time for the next interval
        #             # Reset the start time to measure time per iteration
        #         start_time = time.time()

        # # print("df")

        
        # start_time = current_time
        # df = pd.DataFrame({
        #     "application_number": application_number[:2],
        #     "label": label[:2],

        #     "new_ratio": new_ratio_vec,
        #     "new_bin": new_bin_vec,

        #     "uniq_ratio": uniq_ratio_vec,
        #     "uniq_bin": uniq_bin_vec,

        #     "diff_ratio": diff_ratio_vec,
        #     "diff_bin": diff_bin_vec, 
        #     "neighboroud_distance": neighborhood_distance_vec,
        #     # "mean100": mean100_vec

        #     # "surpNew_ratio": surpNew_ratio,
        #     # "surpNew_bin": surpNew_bin,

        #     "surpDiv_ratio":surpDiv_ratio_vec,
        #     "surpDiv_bin": surpDiv_bin_vec
        # })
        # # current_time = time.time()  # Get the current time
        # # elapsed_time = current_time - start_time
        # # print(elapsed_time)

        # tE_cols_str = "_".join(tE_cols)
        # base_cols_str = "_".join(base_cols)
        # df.to_csv(pathOutput + f'/{year}_{ipc}_{tE_cols_str}_vs_{base_cols_str}_Metrics.csv', index=False)   

2016
    H01L


100%|██████████| 284544/284544 [01:15<00:00, 3779.13it/s]


In [ ]:
# base_texts = [word for text in (ES) for word in text.split()]

# ES_PMI.update(base_texts)
# instance_ES_pmi = ES_PMI.compute_pmi()
# base_bigram_set = set(instance_ES_pmi.keys())

# dict_ES_pmi = pmi_to_dict_adj_dict(instance_ES_pmi)

In [25]:
del base_texts

In [4]:
chunk_size = 10000  # Adjust based on available memory
ES_PMI_2 = OptimizedIncrementalPMI(3)

for i in range(0, len(ES), chunk_size):
    # base_texts = (word for text in ES[i:i+chunk_size] for word in text.split())  # Generator
    base_texts = [word for text in (ES[i:i+chunk_size]) for word in text.split()]
    ES_PMI_2.update(base_texts)
    print(i)

0
10000
20000
30000
40000
50000
60000
70000
80000
90000
100000
110000
120000
130000
140000
150000
160000
170000
180000
190000
200000
210000
220000
230000
240000
250000
260000
270000
280000


In [5]:
instance_ES_pmi_2 = ES_PMI_2.compute_pmi()
base_bigram_set_2 = set(instance_ES_pmi_2.keys())

dict_ES_pmi_2 = pmi_to_dict_adj_dict(instance_ES_pmi_2)

In [6]:
dict_ES_pmi_2 == dict_ES_pmi

False

In [9]:
len(dict_ES_pmi_2)

108188

In [10]:
len(dict_ES_pmi)

108188

In [12]:
dict_ES_pmi_2.keys() == dict_ES_pmi.keys()

True

In [ ]:
dict_ES_pmi_2.values()

In [ ]:
dict